<a href="https://colab.research.google.com/github/YellowRoot/FishLens/blob/main/FishLens.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Imports**

In [3]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from tqdm import tqdm
import numpy as np

# **Loading Dataset**

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# **Import the images**

In [5]:
# File path
dataset = "/content/gdrive/MyDrive/Colab_Notebooks/Datasets/Fish_Dataset"

# Create data generators
train_data = ImageDataGenerator(rescale=1./255, preprocessing_function=tf.keras.applications.vgg16.preprocess_input)
test_data = ImageDataGenerator(rescale=1./255, preprocessing_function=tf.keras.applications.vgg16.preprocess_input)

train_generator = train_data.flow_from_directory(directory=(dataset + "/Training"),
                                                target_size=(224, 224),
                                                batch_size=32,
                                                class_mode='categorical')

test_generator = test_data.flow_from_directory(directory=(dataset + "/Testing"),
                                              target_size=(224, 224),
                                              batch_size=32,
                                              class_mode='categorical')

Found 4500 images belonging to 9 classes.
Found 4500 images belonging to 9 classes.


# **Initialize the model**

In [6]:
# Load pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Add your own classification layers
model = tf.keras.models.Sequential([
    base_model,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(4096, activation='relu'),
    tf.keras.layers.Dense(4096, activation='relu'),
    tf.keras.layers.Dense(9, activation='softmax')
])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

58889256/58889256 [==============================] - 4s 0us/step


# **Model Summary**

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 4096)              102764544 
                                                                 
 dense_1 (Dense)             (None, 4096)              16781312  
                                                                 
 dense_2 (Dense)             (None, 9)                 36873     
                                                                 
Total params: 134,297,417
Trainable params: 119,582,729
Non-trainable params: 14,714,688
_________________________________________________________________


# **Model Training**
### `Implement a callback system to monitor and control training`

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

# Set up callbacks
check = ModelCheckpoint("vgg16_1.h5", monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
stop = EarlyStopping(monitor='val_accuracy', patience=20, verbose=1, mode='max', min_delta=0.001)

# Train the model
num_epochs = 100
batch_size = 32
steps_per_epoch = len(train_generator) // batch_size
validation_steps = len(test_generator) // batch_size

# Use tqdm for progress monitoring
with tqdm(total=num_epochs, unit="epoch") as pbar:
    for epoch in range(num_epochs):
        pbar.set_description(f"Epoch {epoch+1}/{num_epochs}")
        history = model.fit(train_generator, epochs=1, callbacks=[check, stop], validation_data=test_generator, workers=1, use_multiprocessing=False)
        pbar.update(1)

        # Check if validation accuracy reaches the desired level
        if history.history['val_accuracy'][0] >= 0.85:  # Desired accuracy threshold
            print("Validation accuracy reached the desired level. Stopping training.")
            break

Epoch 1/100:   0%|          | 0/100 [00:00<?, ?epoch/s]

 14/141 [=>............................] - ETA: 45:44 - loss: 14.5062 - accuracy: 0.1652

## **Test the Model**

In [ ]:
# Load the trained model
model = tf.keras.models.load_model("vgg16_1.h5")

# Load and preprocess the image
img_path = dataset + "/Predict/1.png"  # Replace with the path to your image
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

# Make predictions
predictions = model.predict(x)
class_labels = ['Black Sea Sprat', 'Gilt-Head Bream', 'Hourse Mackerel', 'Red Mullet', 'Red Sea Bream', 'Sea Brass', 'Shrimp', 'Striped Red Mullet', 'Trout']  # Replace with your class labels

# Print predicted class probabilities
for i, prob in enumerate(predictions[0]):
    print(f"{class_labels[i]}: {prob}")

# Get the predicted class label
predicted_class_index = np.argmax(predictions[0])
predicted_class = class_labels[predicted_class_index]

print("Predicted class:", predicted_class)